<br><br><br>
<h2><font color="#004D7F" size=6>Práctica 2</font></h2>



<h1><font color="#004D7F" size=5> SISTEMAS BASADOS EN REGLAS DIFUSAS DESCRIPTIVAS LINGÜÍSTICAS TIPO MANDANI. APRENDIZAJE DE SISTEMAS PARA CLASIFICACIÓN</font></h1>

<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Guillermo Tomás Fernández Martin y José Miguel Puerta Callejón</font><br>
<font color="#004D7F" size=3>Guillermo.Fernandez@uclm.es</font><br>
<font color="#004D7F" size=3>Sistemas Basados en el Conocimiento</font><br>
<font color="#004D7F" size=3>Grado en Ingeniería Informática (UCLM)</font><br>
</div>

<div style="text-align: left">
<font color="#004D7F" size=4>ESTUDIANTES: </font><br>
<font color="#004D7F" size=4>PABLO LARIO GOMEZ</font><br>
<font color="#004D7F" size=4>DIEGO MIGUEL LOPEZ GARCIA</font><br>
<font color="#004D7F" size=3>nombre de archivo lluvias Australia: 'weatherAUS.csv', directorio: mismo que la libreta</font><br>
</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Introducción](#introduccion)
* [2. Metodología](#meto)
* [3. Implementación](#scikit)
* [4. Resultados](#c45)
* [5. Entrega](#entrega)


---

<br>

<a id="introduccion"></a>
# <font color="#004D7F"> 1. Introducción</font>

En esta práctica vamos a aprender y después utilizar para inferencia un sistema de reglas difusas para clasificación de tipo Mandani basado en variables difusas lingüísticas. Para ello vamos a partir, como siempre, de una base de datos con las variables predictoras y la variable objetivo a predecir. En el caso de esta práctica, la **lluvia en Australia**.

Para entender un poco mejor en que consiste un sistema de reglas difusas, vamos a ver dos ejemplos con otras bases de datos. Primero, un caso donde, a partir de la calidad de la comida y del servicio, decidiremos si dar propina o no. En base a esto, podemos aprender un sistema difuso con reglas de la forma:

<img src="https://drive.google.com/uc?export=view&id=1_s89dvmpugs2Ls-wa4WTY40DL8otf_ZS" width="500">

- Si _CC_ es Buena y _CS_ es Agradable entonces Propina es Si 
- Si _CC_ es Mala y _CS_ es Aceptable entonces Propina es No 
- Si _CC_ es Decente y _CS_ es Aceptable entonces Propina es Si 

<br>

Otro ejemplo, es el que ya conocemos de la calidad del vino: 

<img src="https://drive.google.com/uc?export=view&id=1AXL42lnta40u3XdgGuSAIrAm3ZqfYuKk" width="500">

- Si _NAl_ es Bajo y _NAc_ es Alto entonces Calidad es Mala 
- Si _NAl_ es Medio y _NAc_ es Bajo entonces Calidad es Buena 
- Si _NAl_ es Alto y _NAc_ es Bajo entonces Calidad es Buena 

El objetivo de esta práctica es el de obtener un sistema basado en reglas difusas para la predicción de la lluvia en Australia utilizando árboles de decisión y realizar un proceso de inferencia a partir de dicho sistema. Para ello, vamos a llevar a cabo 3 fases:
1. Realizar una transformación de las variables continuas a variables difusas.
2. Obtener un sistema de reglas basado en un árbol de decisión.
3. Implementar un sistema de inferencia para la obtención de predicciones.

Por último habrá que llevar a cabo una comparativa de los árboles obtenidos en la práctica anterior.



---

<br>

<a id="data"></a>
# <font color="#004D7F"> 2. Metodología</font>


En primer lugar, dada la base de datos hay que definir las etiquetas lingüísticas para cada variable continua de la base de datos, esto es, el número de etiquetas, su nombre semántico y su definición con los conjuntos difusos asociados a cada etiqueta, como, por ejemplo, en los casos anteriores, Nivel de Alcohol, cuyo universo del discurso corre entre 8º hasta 20º, 3 etiquetas, con nombres bajo, medio, alto y definiciones triangulares para medio y trapezoides para bajo y alto.

Para el resto de las variables discretas de nuestra base de datos no hace falta nada más que extraer el número de valores únicos que aparecen en las bases de datos para almacenar y definir sus posibles valores de etiquetas, esto es, si tenemos una variable tipo de vino, pues tendrá solo dos valores: blanco o tinto (si acaso rosado también), con lo que tendremos una variable discreta que pueda tomar esos valores de etiquetas.

Una vez definida esta parte de la base de conocimiento del problema en cuestión, tenemos que pasar a aprender las diferentes reglas de clasificación, para ello procederemos como sigue.

### <font color="#004D7F"> 2.1 Fuzzificación de la Base de Datos</font>

Este paso consiste en transformar los valores numéricos de la base de datos a su etiqueta difusa correspondiente. Para ello, lo que realizaremos es obtener el grado de pertenencia para cada etiqueta definida para la variable y valor en cuestión y quedarnos con la etiqueta con mayor grado de pertenencia. Una vez identificada la etiqueta con mayor grado de pertenencia pasamos a substituir el valor numérico por su etiqueta.




In [1]:
import pandas as pd

In [2]:
pd.DataFrame([[18.5, 7.5, 'Blanco', 'Buena'],[12,12.5,'Tinto','Mala']], columns=['Nivel de Alcohol', 'Nivel de Acidez', 'Tipo', 'Calidad'])

,Nivel de Alcohol,Nivel de Acidez,Tipo,Calidad
0,18.5,7.5,Blanco,Buena
1,12.0,12.5,Tinto,Mala


Se transformaría a la siguiente base de datos:

In [3]:
pd.DataFrame([['Alto', 'Bajo', 'Blanco', 'Buena'],['Bajo','Alto','Tinto','Mala']], columns=['Nivel de Alcohol', 'Nivel de Acidez', 'Tipo', 'Calidad'])

,Nivel de Alcohol,Nivel de Acidez,Tipo,Calidad
0,Alto,Bajo,Blanco,Buena
1,Bajo,Alto,Tinto,Mala


### <font color="#004D7F"> 2.2 Aprendizaje de un Árbol de Decisión</font>

Una vez que tenemos fuzzificada la base de datos podemos interpretarla como una base de datos discreta completamente y, por tanto, podemos utilizarla como entrada para aprender un árbol de decisión con los algoritmos que hemos desarrollado en la práctica anterior.

Esto es, podemos aprender un árbol a partir de la base de datos anterior pasada al valor de etiqueta de cada variable y valor. En el caso anterior podríamos tener un árbol de salida tal como el siguiente:


<img src="https://drive.google.com/uc?export=view&id=1gbA3pg0nvVbQ5YzWKWaZr0s-g6icJ6QM" width="500">

Con este resultado podemos tener las siguientes reglas:
- $r_1$: Si _NAl_ es Bajo entonces Calidad es Mala
- $r_2$: Si _NAl_ es Medio y _NAc_ es Bajo y Tipo es Blanco entonces Calidad es Buena
- $r_3$: Si _NAl_ es Medio y _NAc_ es Bajo y Tipo es Tinto entonces Calidad es Mala
- $r_4$: Si _NAl_ es Medio y _NAc_ es Alto entonces Calidad es Mala
- $r_5$: Si _NAl_ es Alto entonces Calidad es Buena

Fijémonos que las variables discretas en la base de datos original se pueden incorporar en el árbol de forma natural y por tanto en las reglas de salida pueden tener reglas con variables difusas y variables discretas sin ningún problema. En el ejemplo, tenemos la variable Tipo de Vino que es discreta por naturaleza.



### <font color="#004D7F"> 2.3 Inferencia en la clasificación de una instancia</font>

Una vez definido nuestro sistema de reglas difusas lingüísticas para clasificación extraídas del árbol de decisión, tenemos que ver cómo realizar la inferencia para clasificar una nueva instancia.

Esta inferencia se puede realizar directamente en el procesado del árbol, o también utilizando el sistema de reglas extraído. Las dos formas son equivalentes y podrás implementar la que quieras.

#### **Inferencia utilizando el Sistema de Reglas Difusas:**
En este caso se disparan todas las reglas con el ejemplo que tengamos, y calculamos el grado de cobertura que tiene cada una de las reglas. Es decir, multiplicamos el grado de pertenencia de la etiqueta correspondiente a la premisa de la regla para cada premisa. En el caso de tener una premisa discreta, no difusa, se multiplicaría por cero si no corresponden los valores de la premisa con el del ejemplo, o uno si corresponde, es decir es una premisa no difusa, con valores de verdad uno o cero.

La decisión final sobre la salida de la variable clase se realizará de la siguiente forma: cada regla tendrá un grado de cobertura o recubrimiento y un valor de salida para la variable clase, lo notaremos como $r_i:\{C_i,w_i\}$, donde $C_i$ es el valor de la clase y $w_i$ es el grado de cubrimiento de la regla al ejemplo. 

El valor de clase de salida para el ejemplo se calculará cómo la clase mayoritaria ponderada por su valor de recubrimiento o cobertura, es decir, para la clase $C_i$ el número de votos será $∑_{r_j/C_j==C_i}w_j$ , es decir sumamos por cada regla que tenga como salida $C_i$ el valor de su grado de cobertura. Una vez calculado esto para cada etiqueta de salida, nos quedamos con la mayoritaria.

Por ejemplo, imaginad que tenemos la instancia a clasificar: **Nivel de Alcohol = 15, Nivel de Acidez = 5, Tipo = Blanco**. Esperamos ver la Calidad del vino como salida.

- $r_1$: Si NAl es Bajo entonces Calidad es Mala —> $r_1:\{Mala,w_i=0\}; μ_{Bajo}^{NAl} (15)=0$
- $r_5$: Si NAl es Alto entonces Calidad es Buena —> $r_5:\{Buena,w_i=0.33\}; μ_{Alto}^{NAl} (15)=0.33$
- $r_4$: Si NAl es Medio y NAc es Alto entonces Calidad es Mala —> $r_4:\{Mala,w_i=0.22\}; μ_{Medio}^{NAl} (15)=0.66 * μ_{Alto}^{NAc} (5)=0.33 = 0,22$
- $r_3$: Si NAl es Medio y NAc es Bajo y Tipo es Tinto entonces Calidad es Mala —> $r_3:\{Mala,w_i=0\}$. Tiene peso 0 porque Tipo es Blanco y en la regla tenemos Tipo es Tinto.
- $r_2$: Si NAl es Medio y NAc es Bajo y Tipo es Blanco entonces Calidad es Buena —> $r_2:\{Buena,w_i=0.44\}; μ_{Medio}^{NAl} (15)=0.66 * μ_{Bajo}^{NAc} (5)=0.66 * 1 = 0,44$

En este caso tenemos la suma de:
- $Buena = r_2 (0.44) + r_5 (0.33) = 0.77$
- $Mala = r_1(0) +r_4(0.22) +r_3(0) = 0.22$

por lo que se clasificaría como Buena


---

<br>

<a id="data"></a>
# <font color="#004D7F"> 3. Implementación</font>

Para esta práctica el alumno debe llevar a cabo la implementación de las tres etapas descritas en la metodología, con el objetivo final de crear un sistema de reglas difusas para el problema de la predicción de la lluvia en Australia y ser capaz de realizar predicciones a partir de la inferencia. Se utilizará el paquete `scikit-fuzzy`, que podéis instalar con `pip install scikit-fuzzy`.

Primero hay que realizar el proceso de Fuzzificación de la base de datos de la lluvia en Australia, donde las variables continuas deben ser convertidas a etiquetas. Después hay que llevar a cabo el proceso de aprendizaje del árbol utilizando la implementación realizada en la práctica 1. Por último, a partir de este árbol, el alumno debe de implementar un método para llevar a cabo la inferencia a partir de la base de datos original.

Vamos a describir un poco más en detalle cada uno de estos pasos:



### <font color="#004D7F"> 3.1 Variables Difusas</font>

El primer paso será generar las variables difusas a partir de las variables de la base de datos. Para ello, se proporciona el esqueleto de una clase que os ayudará a estructurar las variables con los elementos necesarios para la transformación posterior de la base de datos. Para simplificar, se utilizarán conjuntos difusos triangulares.

Los atributos de la clase serán los siguientes:
- `universe`: El universo del discurso de la variable. **IMPORTANTE** `scikit-fuzzy` asume que el universo del discurso está ordenado para todas sus operaciones. Sin embargo, cuando extraemos una columna de la base de datos para generar la variable difusa no tiene por qué estar ordenada. Habrá que ordenarla para poder utilizar las funciones de `scikit-fuzzy` pero sin modificar la base de datos original.
- `memb_func`: Un diccionario con los grados de pertenencia de cada elemento del universo del discurso para cada uno de los conjuntos difusos que se generen de la variable difusa
- `labels`: Etiquetas de cada uno de los conjuntos difusos
- `name`: Nombre de la variable difusa

Las funciones de la clase serán las siguientes:
- `_membership_functions`: Función interna para generar el diccionario con las distintas funciones de pertenencia. A COMPLETAR
- `fuzzify`: Función que devuelve el mejor conjunto difuso para un valor dentro del universo del discurso. A COMPLETAR
- `plot`: Función que muestra de manera gráfica los conjuntos difusos de la variable. Ya viene dada.

Para utilizar la clase, habrá que calcular previamente los triángulos de los conjuntos difusos. Para ello, se utilizará la estructura `fuzzy_points`, que es diccionario con la definición de los triángulos y forma `{label: [start, peak, end]}` para cada triángulo.

Para generar esta estructura `fuzzy_points` se recomienda calcular las particiones de las variables de manera automática, ya sea calculando por anchura o por frecuencia. `Pandas` tiene funciones que podéis buscar para discretizar una variable tanto por anchura como por frecuencia, se recomienda utilizarlas.

In [4]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pandas as pd


class FuzzyVariable:
    def __init__(self, name, variable, fuzzy_points):
        # DAR LAS VARIABLES PERO SIN RELLENAR
        self.universe = np.sort(df[variable].to_numpy())
        self.labels = ['Low', 'Mid', 'High']
        self.memb_func = self._membership_functions(variable, fuzzy_points)
        self.name = name
        self.fuzzyPoints = fuzzy_points
    
    def _membership_functions(self, variable, fuzzy_points):
        fuzz_dict = {}

        # GENERAR LOS DICCIONARIOS CON LAS FUNCIONES DE PERTENENCIA
        aux = np.arange(len(self.universe))
        for i in range(len(aux)):
            aux[i] = 0

        fuzz_dict['Low'] = aux
        fuzz_dict['Mid'] = aux
        fuzz_dict['High'] = aux

        # fuzz_dict['Low'] = fuzz.trimf(self.universe, fuzzy_points['Low'])
        # fuzz_dict['Mid'] = fuzz.trimf(self.universe, fuzzy_points['Mid'])
        # fuzz_dict['High'] = fuzz.trimf(self.universe, fuzzy_points['High'])

        return fuzz_dict
    
    def fuzzify(self, value):
        best = ''
        max_val = -1

        # for label in self.labels: # Iterar por cada variable para ver la mejor
        #     # DEVOLVER EL CONJUNTO DIFUSO QUE MAYOR GRADO DE PERTENENCIA TENGA
        #     pass

        p1 = self.fuzzyPoints["Low"][1]
        p2 = self.fuzzyPoints["Mid"][1]
        p3 = self.fuzzyPoints["High"][1]
        
        best = fuzzificar(value, p1, p2, p3)

        return best

    def plot(self):
        fig, ax0 = plt.subplots(nrows=1)
        for label in self.labels:
            ax0.plot(self.universe, self.fuzz_dict[label], linewidth=1.5, label=label)
        ax0.set_title(self.name)
        ax0.legend()
        plt.tight_layout()

### <font color="#004D7F"> 3.2 Transformador Fuzzy</font>

Para llevar a cabo la Fuzzificación de la base de datos, el alumno implementará un Transformador que se encargue de generar las variables difusas a partir de las variables continuas. La implementación consiste en generar las variables difusas necesarias y utilizar la función `fuzzify` para fuzzificar los valores. Se recomienda estudiar la función `DataFrame.apply` para realizar esta transformación

Los atributos de la clase son los siguientes:
- `sets`: Número de conjuntos difusos para cada variable, por defecto 3
- `labels`: Etiquetas difusas para los conjuntos. **DEBE TENER LA MISMA LONGITUD QUE `sets`**

La clase se dividirá en las funciones:
- `fit`: encargado de aprender las variables difusas a partir de los datos
- `transform`: creará una nueva base de datos sustituyendo los valores continuos por las etiquetas de las variables
- `fit_transform`: auxiliar para hacer los dos procedimientos en un solo paso.
- `_equal_width`: Esta función coge una variable en formato `NumPy` y genera cortes de igual anchura. Es decir, si la variable es `[1,2,3,4,5,6,7]` y tiene que hacer 3 cortes, devuelve los puntos `[1,4,7]`
- `_expand_points`: Esta función coge los puntos de `equal_width` y los expande para generar los triángulos en el formato `[start, peak, end]`

In [5]:
from sklearn.base import BaseEstimator


# Ejemplo de la estructura básica de un algoritmo que hereda de BaseEstimator siguiendo la estructura de SciKit
class FuzzyEncoder(BaseEstimator):
    # constructor
    def __init__(self, sets=3, labels=['Low', 'Mid', 'High']):
        self.fuzzy_vars = {}
        self.sets = sets
        self.labels = labels
        self.aux_fuzzy_vars = {}

    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X):

        # APRENDER LAS VARIABLES DIFUSAS A PARTIR DE LOS DATOS

        for i in range(len(X.columns)):

            columna = X.columns[i:i+1]
            array = X[columna].to_numpy()

            arrayUnidimensional = np.reshape(array, (1, np.product(array.shape)))[0] # Sacar un array unidimensional

            if (type(arrayUnidimensional[0]) == str):
                continue

            equal_width_points = self._equal_width(arrayUnidimensional, 3)  # Mismo que en el ejemplo

            fuzzy_points = self._expand_points(equal_width_points, ['Low', 'Mid', 'High'])  # Mismo que en el ejemplo

            f = FuzzyVariable(columna[0], columna[0], fuzzy_points)
            self.aux_fuzzy_vars[columna[0]] = {
                'universe' : f.universe,
                'pert': {l: f.memb_func[l] for l in f.labels}
            }

            self.fuzzy_vars[columna[0]] = equal_width_points

        return self
        
    # y transform para aplicar lo aprendido y transformar la base de datos creando una nueva
    def transform(self, X):
        
        new_X = X.copy()

        # MODIFICAR new_X CON LAS VARIABLES DIFUSAS APRENDIDAS

        for columna in self.fuzzy_vars:
            
            equal_width_points = self.fuzzy_vars[columna]
            
            new_X[columna] = X[columna].apply(fuzzificar, args=equal_width_points)

        return new_X
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def _equal_width(self, var, sets):
        try:
            cut = pd.cut(var, sets - 1)
            sol = [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        except ValueError: 
            cut = pd.cut(var, sets - 1, duplicates='drop')
            sol = [var.min()] + [cat.left for cat in cut.categories] + [cut.categories[-1].right]
        sol[0] = var.min()
        return sol

    def _expand_points(self, fp, labels):
        ep = {}
        expanded_points = [fp[0]] + fp + [fp[-1]]

        for i in range(len(expanded_points) - 2):
            ep[labels[i]] = [expanded_points[i], expanded_points[i+1], expanded_points[i+2]]

        return ep

Se ha de realizar el mismo preprocesamiento que en la práctica 1 con dos nuevas restricciones:
- La variable `Location` hay que eliminarla. Esto lo vamos a hacer para que el árbol que salga no ramifique demasiado
- Las variables de puntos cardinales hay que simplificarlas. Se han de quedar simplemente con los cuatro puntos cardinales principales (N, S, E, W)

<h1><font color="#004D7F" size=5>PROCESADO DE BASE DE DATOS</font></h1>

In [6]:
df = pd.read_csv('weatherAUS.csv')

# Eliminamos todos los valores nulos

# Variables continuas
df['MinTemp'] = df['MinTemp'].fillna(df['MinTemp'].mean())
df['MaxTemp'] = df['MaxTemp'].fillna(df['MaxTemp'].mean())
df['Rainfall'] = df['Rainfall'].fillna(df['Rainfall'].mean())
df['WindGustSpeed'] = df['WindGustSpeed'].fillna(df['WindGustSpeed'].mean())
df['WindSpeed9am'] = df['WindSpeed9am'].fillna(df['WindSpeed9am'].mean())
df['WindSpeed3pm'] = df['WindSpeed3pm'].fillna(df['WindSpeed3pm'].mean())
df['Humidity9am'] = df['Humidity9am'].fillna(df['Humidity9am'].mean())
df['Humidity3pm'] = df['Humidity3pm'].fillna(df['Humidity3pm'].mean())
df['Pressure9am'] = df['Pressure9am'].fillna(df['Pressure9am'].mean())
df['Pressure3pm'] = df['Pressure3pm'].fillna(df['Pressure3pm'].mean())
df['Cloud9am'] = df['Cloud9am'].fillna(df['Cloud9am'].mean())
df['Cloud3pm'] = df['Cloud3pm'].fillna(df['Cloud3pm'].mean())
df['Temp9am'] = df['Temp9am'].fillna(df['Temp9am'].mean())
df['Temp3pm'] = df['Temp3pm'].fillna(df['Temp3pm'].mean())

# Variables categóricas
df['WindGustDir'].replace(np.nan, 'W', inplace=True)
df['WindDir9am'].replace(np.nan, 'N', inplace=True)
df['WindDir3pm'].replace(np.nan, 'SE', inplace=True)
df['RainToday'].replace(np.nan, 'No', inplace=True)
df['RainTomorrow'].replace(np.nan, 'No', inplace=True)

# Simplificamos en solo los cuatro puntos cardinales
df['WindGustDir'].replace('WNW', 'W', inplace=True)
df['WindGustDir'].replace('WSW', 'W', inplace=True)
df['WindGustDir'].replace('NE', 'N', inplace=True)
df['WindGustDir'].replace('NNW', 'N', inplace=True)
df['WindGustDir'].replace('NNE', 'N', inplace=True)
df['WindGustDir'].replace('SSE', 'S', inplace=True)
df['WindGustDir'].replace('NW', 'N', inplace=True)
df['WindGustDir'].replace('SW', 'S', inplace=True)
df['WindGustDir'].replace('ENE', 'E', inplace=True)
df['WindGustDir'].replace('SE', 'S', inplace=True)
df['WindGustDir'].replace('ESE', 'E', inplace=True)
df['WindGustDir'].replace('SSW', 'S', inplace=True)

df['WindDir9am'].replace('NNW', 'N', inplace=True)
df['WindDir9am'].replace('SE', 'S', inplace=True)
df['WindDir9am'].replace('ENE', 'E', inplace=True)
df['WindDir9am'].replace('SW', 'S', inplace=True)
df['WindDir9am'].replace('SSE', 'S', inplace=True)
df['WindDir9am'].replace('NE', 'N', inplace=True)
df['WindDir9am'].replace('SSW', 'S', inplace=True)
df['WindDir9am'].replace('WSW', 'W', inplace=True)
df['WindDir9am'].replace('ESE', 'E', inplace=True)
df['WindDir9am'].replace('NW', 'N', inplace=True)
df['WindDir9am'].replace('WNW', 'W', inplace=True)
df['WindDir9am'].replace('NNE', 'N', inplace=True)

df['WindDir3pm'].replace('WNW', 'W', inplace=True)
df['WindDir3pm'].replace('WSW', 'W', inplace=True)
df['WindDir3pm'].replace('NW', 'N', inplace=True)
df['WindDir3pm'].replace('SSE', 'S', inplace=True)
df['WindDir3pm'].replace('ESE', 'E', inplace=True)
df['WindDir3pm'].replace('ENE', 'E', inplace=True)
df['WindDir3pm'].replace('NNW', 'N', inplace=True)
df['WindDir3pm'].replace('SSW', 'S', inplace=True)
df['WindDir3pm'].replace('SW', 'S', inplace=True)
df['WindDir3pm'].replace('SE', 'S', inplace=True)
df['WindDir3pm'].replace('NNE', 'N', inplace=True)
df['WindDir3pm'].replace('NE', 'N', inplace=True)

encoder = LabelEncoder()
df['RainTomorrow'] = encoder.fit_transform(df['RainTomorrow'])

features = df.columns
attributes = features[2:-1] # La variable Location queda eliminada
target = features[-1]

X = df[attributes]
y = df[target]

discrete = []
for columna in X:
    if isinstance(X[columna][0], str):
        discrete.append(columna)

<h1><font color="#004D7F" size=5>FUZZIFICACION DE BASE DE DATOS</font></h1>

In [7]:
def fuzzificar(value, p1, p2, p3):

    a = (p1 + p2) / 2
    b = (p2 + p3) / 2

    if value < a:
        return "Low"
    elif value < b:
        return "Mid"
    else:
        return "High"

In [8]:
fuzzy_vars = FuzzyEncoder()
fuzzy_vars.fit(X)
fuzzy_X = fuzzy_vars.transform(X)
fuzzy_X

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Mid,Mid,Low,High,High,W,Mid,W,W,Low,Mid,Mid,Low,Mid,Mid,High,Mid,Mid,Mid,No
1,Mid,Mid,Low,High,High,W,Mid,N,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid,No
2,Mid,Mid,Low,High,High,W,Mid,W,W,Low,Mid,Mid,Mid,Mid,Mid,Mid,Low,Mid,Mid,No
3,Mid,Mid,Low,High,High,N,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
4,Mid,Mid,Low,High,High,W,Mid,E,N,Low,Low,High,Mid,Mid,Mid,High,High,Mid,Mid,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Mid,Mid,Low,High,High,E,Low,S,E,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145456,Mid,Mid,Low,High,High,N,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145457,Mid,Mid,Low,High,High,N,Low,S,W,Low,Low,Mid,Low,Mid,Mid,Mid,Mid,Mid,Mid,No
145458,Mid,Mid,Low,High,High,S,Low,S,N,Low,Low,Mid,Low,Mid,Mid,Mid,Low,Mid,Mid,No


### <font color="#004D7F"> 3.3 Árbol C4.5</font>
En esta parte el alumno debe aprender un árbol utilizando la nueva base de datos fuzzificada y la implementación realizada en la práctica 1. Para ello, debe corregir los errores, si los hubiera, de la entrega anterior.


In [9]:
import numpy as np
from scipy.stats import entropy
from sklearn.base import BaseEstimator

class Tree_c45:
    def __init__(self):
        # primero indicamos si el objeto es hoja o raiz
        self.is_leaf = True
        self.cat_dict = None
        
        # atributos para el corte cuando el objeto es una raiz
        # indicamos el indice de la variable de corte y los valores
        self.var = None
        self.var_index = -1
        self.cut_value = 0
        # los hijos deben ser objetos de Tree_c45
        self.list_of_childs = []
        
        # atributos cuando el Tree_c45 es una hoja
        # indicamos el valor de la clase y en class_count una tupla (casos con ese valor, casos totales en la hoja)
        self.class_value = -1
        self.class_count = 0
        
        # para el nivel de profundidad en el arbol de la raiz o la hoja
        self.level = -1
        
    def __str__(self):
        output = ''
        if(self.is_leaf):
            output += 'Class value: ' + str(self.class_value) + '\tCounts: ' + str(self.class_count)
        else:
            output += 'Feature '+ str(self.var)
            for i in range(len(self.list_of_childs)):
                output += '\n'+'\t'*(self.level+1)+str(self.cut_value[i])+': '+str(self.list_of_childs[i]) 
            
        return output
    
    # esta funcion nos servira para hacer predicciones y debe ser completada
    def predict(self,x):
        if self.is_leaf:
            return self.class_value
        else:
            child_index = self.cat_dict[x[self.var_index]]
            return self.list_of_childs[child_index].predict(x)
            

class C45(BaseEstimator):
    # constructor con la profundidad del arbol, que por defecto es 2 y donde inicializamos el arbol que vamos a aprender
    def __init__(self, features=None, max_depth=2, debug=False):
        self.max_depth = max_depth
        self.features = features
        self.features_dict = {col: i for i, col in enumerate(features)}
        self.tree = Tree_c45()
        self.debug = debug
        
    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X, y):
        # esta será la llamada a la funcion recursiva que aprendera el arbol a partir de los datos
        self.make_cut(X, y, self.tree, 0, set([]))
        
    # y predict para las predicciones de casos nuevos
    def predict(self, X):
        return np.array([self.tree.predict(x) for x in X])
    
    # tambien se da una funcion score que calcula el accuracy
    def score(self, X, y):
        return np.sum(self.predict(X)==y)/y.shape[0]

    def _get_cat_entropy(self, var, target, class_values):
        join = np.vstack((var, target)).T
        join = join[join[:, 0].argsort()]
        spl = np.unique(var)
        e = 0

        for x in spl:
            splar = join[join[:, 0] == x, 1].astype('int')
            cummulative = []

            for j in class_values:
                cummulative.append(np.sum(splar == j) / splar.shape[0])

            e += splar.shape[0] / join.shape[0] * entropy(cummulative, base=2)
        
        return e, spl
    
    # esta es la funcion principal, encargada de encontrar la variable y su punto de corte que maximice la ganancia en base a la entropia.
    # se basa en que los cortes se van a realizar sobre variables continuas aunque la salida sea discreta
    def make_cut(self, X, y, current_tree, current_depth, borradas):

        # esqueleto con la funcionalidad basica
        
        # antes de realizar ninguna operacion comprobamos si hemos alcanzado el limite del arbol,
        # si es asi, actualizamos el arbol ya que estamos en un nodo hoja 
        # y no hace falta buscar variable ni continuar con las llamadas recursivas
        if current_depth >= self.max_depth:
            # TO DO
            current_tree.is_leaf = True
            unique, counts = np.unique(y, return_counts=True)
            m = np.argmax(counts)
            current_tree.class_value = np.amax(unique[m])
            current_tree.class_count = [len(y[y == m]), len(y)]
            return

        # primero, obtenemos la entropia de la clase
        # TO DO        
        
        class_values = [0,1]
        num_cases = len(X)
        frecuency_list = []

        for i in class_values:
            actual_class_c = np.sum(y==i)
            frecuency_list.append(actual_class_c/num_cases)
            
        class_ent = entropy(frecuency_list, base=2)

        best_gain = -1
        best_var = None
        best_spl = None

        for col in self.features:

            if col not in borradas:

                var = X[col].to_numpy()
                ent, spl = self._get_cat_entropy(var, y, [0,1])
                gain = class_ent - ent
                
                if gain > best_gain:
                    best_gain = gain
                    best_var = col
                    best_spl = spl
        
        if self.debug:
            print(f'Best variable is {best_var}')

            
        # antes de llamar a la funcion recursiva hay que actualizar los valores del arbol
        # TO DO
        nborradas = borradas.copy()
        nborradas.add(best_var)
        current_tree.is_leaf = False
        current_tree.level = current_depth
        current_tree.cut_value = best_spl
        current_tree.var = best_var
        current_tree.var_index = self.features_dict[best_var]
        
        current_tree.cat_dict = {feat: i for i, feat in enumerate(X[best_var].unique())}
        for i, feat in enumerate(X[best_var].unique()):
            child = Tree_c45()
            current_tree.is_num = False
            ind = X[X[best_var] == feat].index
            current_tree.list_of_childs.append(child)
            self.make_cut(X.loc[ind], y.loc[ind], child, current_depth + 1, nborradas)

El primer paso es entrenar el árbol con las nuevas variables fuzzificadas como si de variables discretas se tratasen

In [10]:
# Probamos a entrenar un árbol de profunidad 2 para ver que funciona el algoritmo y la fuzzificación
t = C45(fuzzy_X.columns)
t.max_depth = 2
t.fit(fuzzy_X, y)
print(t.tree)

Feature Humidity3pm
	High: Feature WindDir3pm
		E: Class value: 0	Counts: [3509, 3655]
		N: Class value: 0	Counts: [2671, 2719]
		S: Class value: 0	Counts: [4630, 5003]
		W: Class value: 0	Counts: [4282, 4338]
	Low: Feature Cloud3pm
		High: Class value: 0	Counts: [58336, 69580]
		Low: Class value: 0	Counts: [19492, 20837]
		Mid: Class value: 0	Counts: [13961, 19904]
	Mid: Feature RainToday
		No: Class value: 1	Counts: [7585, 9965]
		Yes: Class value: 1	Counts: [5137, 9459]


Vamos a utilizar `scikit-fuzzy` para el sistema de control. Eso significa que se necesita un procesado previo para las variables discretas. Se necesitan dos elementos para crear un sistema basado en reglas con `scikit-fuzzy`:
- Universo del discurso. Para una variable discreta, será una enumeración de sus valores. Esto quiere decir que una variable `['N', 'S', 'E', 'W']` tendrá como universo del discurso `[0,1,2,3]`
- Una función de grados de pertenencia. En lugar de utilizar `fuzz.trapmf` o `fuzz.trimf`, la función será `1` cuando coincida con el elemento y `0` cuando no coincida. Para la variable anterior, 
  - `pert['N'] = [1,0,0,0]`,   
  - `pert['E'] = [0,0,1,0]`, 
  - etc.

Se recomienda crear un diccionario para cada una de las variables discretas para luego la generación de las reglas

In [11]:
def get_discrete(X, var):

    universeArray = None

    if var == 'RainTomorrow':
        universeArray = X.unique()
    else:
        universeArray = X[var].unique()

    # Creamos el universo de la variable
    universe = np.array(universeArray)
    universe = np.array(range(len(universe)))

    pert = {}
    i = 0
    for uniqueValue in universeArray:

        array = np.array(range(len(universe)))
        array = np.zeros_like(array)

        # Creamos la función de pertenencia de la variable
        pert[uniqueValue] = array
        pert[uniqueValue][i] = 1
        i += 1

    return {
        'universe': universe,
        'pert': pert
    }

Con este generador, dada la función `get_discrete` que nos genera las variables discretas, se consigue un diccionario por cada una de las variables discretas

In [12]:
# Calculamos las variables discretas
discrete_variables = { att: get_discrete(X, att) for att in discrete }
discrete_variables

{'WindGustDir': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir9am': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'N': array([0, 1, 0, 0]),
   'S': array([0, 0, 1, 0]),
   'E': array([0, 0, 0, 1])}},
 'WindDir3pm': {'universe': array([0, 1, 2, 3]),
  'pert': {'W': array([1, 0, 0, 0]),
   'E': array([0, 1, 0, 0]),
   'N': array([0, 0, 1, 0]),
   'S': array([0, 0, 0, 1])}},
 'RainToday': {'universe': array([0, 1]),
  'pert': {'No': array([1, 0]), 'Yes': array([0, 1])}}}

In [13]:
# Calculamos la variable objetivo
target_variable = {}
target_variable['RainTomorrow'] = get_discrete(y, 'RainTomorrow')
target_variable

{'RainTomorrow': {'universe': array([0, 1]),
  'pert': {0: array([1, 0]), 1: array([0, 1])}}}

De igual manera, hay que crear un diccionario con la variable clase, que también es discreta. Se crea igual que el resto de variables discretas, pero se aconseja pasar esta variable por separado. 

Finalmente se crea el sistema basado en reglas difusas. Vamos a tener una clase abstracta general y una clase que extienda de esta y utilice el árbol de decisión para obtener las reglas.

La clase abstracta `RuleBasedSystem` se dividirá en las funciones:
- `fit`: Metodo abstracto que se programará en la clase que extienda de `RuleBasedSystem`
- `_create_rbs`: encargado de aprender el sistema basado en reglas. Tiene los siguientes parámetros:
  - `rule_list`: Lista de reglas con las que se creará el sistema de control
  - `fuzzy_variables`: Variables difusas extraidas de `FuzzyEncoder.fuzzy_vars`
  - `discrete_variables`: Diccionario con las variables discretas.
  - `target_variable`: Diccionario con la variable objetivo.
- `predict`: Devuelve la clasificación para una instancia de la base de datos segun el sistema basado en reglas
- `score`: Devuelve el score de la base de datos

In [14]:
from skfuzzy import control as ctrl
from abc import ABC, abstractmethod

class RuleBasedSystem(ABC):
    def __init__(self):
        self.ctrl = None

    def _create_rbs(self, rule_list, fuzzy_variables, discrete_variables, target_variable):

        # PASO 1
        
        # Hay que obtener todos los antecedentes. Un antecedente en 
        # scikit-fuzzy se crea con ante = ctrl.Antecedent(universe, variable_name)
        # Para ello, tanto en fuzzy_variables como en discrete_variables
        # deberia estar ya representado el universo de cada variable

        # Ademas, para cada antecedente hay que crear todos los conjuntos difusos
        # (o valores para las variables discretas) correspondientes. Tenemos las etiquetas
        # en las fuzzy_variables, y se pueden obtener tambien de las variables discretas
        # Para cada conjunto difuso se crea con ante[label] = pert, donde ya tenemos
        # las funciones de pertenencia tanto en fuzzy_variables como en discrete_variables

        antecedentes = []

        for discreteVariable in discrete_variables.keys():

            ante = ctrl.Antecedent(discrete_variables[discreteVariable]['universe'], discreteVariable)

            for dvdv in discrete_variables[discreteVariable]['pert']:
                #print("dvdv: " + str(dvdv))
                ante[dvdv] = discrete_variables[discreteVariable]['pert'][dvdv]
            
            antecedentes.append(ante)

        for fuzzyVariable in fuzzy_variables.aux_fuzzy_vars:

            ante = ctrl.Antecedent(fuzzy_variables.aux_fuzzy_vars[fuzzyVariable]['universe'], fuzzyVariable)

            for label in fuzzy_variables.labels:
                ante[label] = fuzzy_variables.aux_fuzzy_vars[fuzzyVariable]['pert'][label]

            antecedentes.append(ante)

        # PASO 2

        # Para crear el consecuente se hace con conseq = ctrl.Consequent(universe, conseq_name)
        # Por eso es mejor pasarlo por separado. De igual manera, hay que asignar cada uno 
        # de los valores que puede tomar y sus funciones de pertenencia

        conseq = ctrl.Consequent(target_variable['RainTomorrow']['universe'], 'RainTomorrow')
        conseq['No'] = target_variable['RainTomorrow']['pert'][0]
        conseq['Yes'] = target_variable['RainTomorrow']['pert'][1]

        rules = []

        # PASO 3

        # Finalmente hay que generar las reglas. Para ello se crean con 
        # rule = ctrl.Rule(antecedent=antecedents, consequent=consequent)
        # Aqui trabajamos con los antecedentes y el consecuente que hemos generado
        # anteriormente. Con la lista de reglas que tenemos del arbol, generamos
        # las reglas con el formato de scikit-fuzzy. antecedents es una union logica 
        # de terminos, esto quiere decir, ante1[label1] & ante2[label2]...
        # De igual manera, el consecuente es conseq[label] para el valor correspondiente
        # de la regla. Habra que haber almacenado los antecedentes para poder acceder
        # a ellos en funcion del nombre de la columna que aparece en la regla

        for regla in rule_list:

            antedecentes = []

            for antedecente in antecedentes:

                for a in regla[0]:

                    if antedecente.label == a[0]:
                        antedecentes.append(antedecente[a[1]])

            consecuente = None
            if regla[1] == 0:
                consecuente = conseq['No']
            else:
                consecuente = conseq['Yes']

            rule = ctrl.Rule(self.conseguirAntecendentesRecursivamente(len(antedecentes), antedecentes), consecuente)
            rules.append(rule)

        for rule in rules:
            print(rule)
            print("")

        self.ctrl = ctrl.ControlSystem(rules=rules)


    def conseguirAntecendentesRecursivamente(self, len, antecedentes):

        # Si la longitud es 1, entonces el método para y devuelve el último antecedente que queda
        if len == 1:
            return antecedentes[0]

        # Eliminamos el primer antecedente y pasamos el resto como parámetro
        aux = []
        for a in antecedentes:

            if a == antecedentes[0]:
                continue

            aux.append(a)

        # Devolvemos la operación lógica entre el antecedente y el resto de antecedentes, que se calcularán recursivamente.
        return antecedentes[0] & self.conseguirAntecendentesRecursivamente(len - 1, aux)

    @abstractmethod
    def fit():
        pass
    
    def predict(self, x):
        sim = ctrl.ControlSystemSimulation(self.ctrl, flush_after_run=1000)
        # Para predecir, hay que añadir los valores de la instancia al sistema
        # de simulacion de control. Primero hay que ver si existe el antecedente 
        # en el sistema de control (posiblemente las reglas extraidas del arbol
        # no tengan todas las caracteristicas de la instancia). En caso afirmativo,
        # la variable 'temperatura' = 8.7 se añade como sim.input['temperatura'] = 8.7    

        return None

        # Despues se calculan los grados de las reglas
        sim.compute()

        # Finalmente se devuelve el resultado que se encuentra en sim.output['target']
        # si la variable a predecir se llama 'target'.

        return sim.output['RainTomorrow']
    

    def score(self, X, y):
        all_predictions = np.array([round(self.predict(x)) for x in X])
        return np.sum(all_predictions==y)/y.shape[0]

La clase `TreeRuleBasedSystem` se dividirá en las funciones:
- `_extract_rules`: encargado de procesar el arbol ya entrenado para extraer las reglas. Se proporciona un formato sugerido para extraer las reglas pero se puede utilizar el formato preferido.
- `fit`: Metodo que entrena el sistema basado en reglas

In [15]:
class TreeRuleBasedSystem(RuleBasedSystem):
    def __init__(self):
        super().__init__()

    # Se recomienda este formato para extraer las reglas
    # del arbol, ya que simplifica la manera de tratar los 
    # antecedentes y consecuentes a la hora de generar el 
    # sistema basado en reglas. A continuación se da 
    # una función con el formato, que habra que modificar para
    # que extraiga adecuadamente las reglas
    
    # extraerReglasRecursivamente es un método recursivo que atraviesa el árbol construyendo los antecedentes y consecuentes correctos.
    def extraerReglasRecursivamente(self, tree, antedecentes, reglas):

        if tree.is_leaf: # Si se llega a un nodo hoja, se calcula el consecuente y se crea la regla

            conse = tree.class_value
            rule = (antedecentes, conse)
            reglas.append(rule)
            return

        else: # Si se llega a un nodo que no se un nodo hoja, se calcula para cada hijo del nodo el nuevo antecedente, y se llama recursivamente a la función.

            i = 0
            for child in tree.list_of_childs:
                ante = (tree.var, tree.cut_value[i])
                nuevosAntedecentes = antedecentes.copy()
                nuevosAntedecentes.append(ante)
                i += 1
                self.extraerReglasRecursivamente(child, nuevosAntedecentes, reglas)

            return 

    def _extract_rules(self, tree):

        reglas = []
        antecedentes = []

        self.extraerReglasRecursivamente(tree.tree, antecedentes, reglas)

        return reglas


    def fit(self, tree, fuzzy_variables, discrete_variables, target_variable):
        
        rule_list = self._extract_rules(tree)

        self._create_rbs(rule_list, fuzzy_variables, discrete_variables, target_variable)

---

<br>

<a id="data"></a>
# <font color="#004D7F"> 4. Resultados</font>

Para finalizar esta práctica, el alumno deberá ejecutar el árbol con diferentes profundidades y comparar los resultados con las versiones discreta y continua de la práctica 1. Se recomienda probar por lo menos profundidad 2, 3 y 4. Además habrá que comparar el rendimiento del árbol tanto en score como en tiempos de ejecución con el rendimiento del sistema basado en reglas correspondiente.

En esta ocasión no habrá que llevar a cabo un estudio tan exhaustivo como en la práctica 1. Simplemente habrá que comparar aquellas variables más importantes y observar si en esta versión difusa el comportamiento es el mismo o difiere.

Opcionalmente, se pueden obtener otros resultados como el tiempo de ejecución o una comparativa del tamaño de los árboles, número de reglas, etc. 


---

<br>

<a id="data"></a>
# <font color="#004D7F"> 5. Entrega</font>

Al igual que en la práctica anterior, el alumno subirá a campus virtual un notebook con la implementación del sistema de reglas y del proceso de inferencia.

Para aprobar la práctica será necesario que la implementación del algoritmo sea correcta, por lo que la parte correspondiente al árbol de decisión de la práctica 1 también debe funcionar. Se tendrán en cuenta para la nota la eficiencia de la implementación, así como la limpieza y los comentarios en el código.

La parte de resultados será necesaria llevarla a cabo para optar a la nota máxima, mientras que la parte opcional será valorada positivamente.

La entrega de esta práctica tiene de plazo hasta el día 24 de Abril de 2022.

### Explicaciones Necesarias

Las tres fases de la práctica consisten en:
- Primero, hemoz fuzzificado la base de datos.
- Segundo, hemos aprendido el árbol, usando el algoritmo provisto por Guillermo en el campus virtual. Aún así, este árbol estaba incompleto, y hemos encesitado modificar algunos cosas para poder utilizarlo en este práctica, ya que al calcular los puntos de corte y la entropía, no se guardaban los puntos de corte, y cuando se calculaba el árbol, en vez del punto de corte aparecía None.
- Tercero, hemos extraído las reglas del árbol y las hemos creado con scikit-fuzzy

Parar lograr fuzzificar la base de datos, hemos usado la siguiente función, que coge dos puntos y encuentra el punto medio entre ambos. Al ser figuras simples (triángulos) y ser rectas entre puntos, podemos calcular fácilmente si un valor pertenece a una parte u a otra. En este caso, si el valor es menor que el punto medio calculado, pertenece a la variable de la izquierda, y si no a la de la derecha.

In [16]:
def fuzzify(value, p1, p2, p3):

    a = (p1 + p2) / 2
    b = (p2 + p3) / 2

    if value < a:
        return "Low"
    elif value < b:
        return "Mid"
    else:
        return "High"

Para lograr extraer las reglas del árbol, hemos usado un algoritmo recursivo que recorre nada nodo y sus hijos.

In [17]:
def extractRulesRecursive(self, tree, antedecentes, reglas):

    if tree.is_leaf: # Si se llega a un nodo hoja, se calcula el consecuente y se crea la regla

        conse = tree.class_value
        rule = (antedecentes, conse)
        reglas.append(rule)
        return

    else: # Si se llega a un nodo que no se un nodo hoja, se calcula para cada hijo del nodo el nuevo antecedente, y se llama recursivamente a la función.

        i = 0
        for child in tree.list_of_childs:
            ante = (tree.var, tree.cut_value[i])
            nuevosAntedecentes = antedecentes.copy()
            nuevosAntedecentes.append(ante)
            i += 1
            self.extraerReglasRecursivamente(child, nuevosAntedecentes, reglas)

        return 

Para poder crear las reglas con sci-kit fuzzy, esta librearía necesita que los antecedentes se junte con el operador lógico &. Para que no importe el número de antecedentes y funcione para cualquier árbol, hemos decidido crear un método recursivo que realizar esta operación para todos los antecedentes.

In [18]:
def getAntecedentesRecursive(self, len, antecedentes):

    if len == 1:
        return antecedentes[0]

    aux = []
    for a in antecedentes:

        if a == antecedentes[0]:
            continue

        aux.append(a)

    return antecedentes[0] & self.conseguirAntecendentesRecursivamente(len - 1, aux)

Vamos ahora a ver los árboles de profundidad 1 hasta 5, y la extracción de sus respectivas reglas.

### Árbol de Profundidad 1

In [19]:
%%time 
t = C45(fuzzy_X.columns)
t.max_depth = 1
t.fit(fuzzy_X, y)
print("ARBOL")
print(t.tree)
print("")
print("")

print("REGLAS")
arbol = TreeRuleBasedSystem()
arbol.fit(t, fuzzy_vars, discrete_variables, target_variable)

ARBOL
Feature Humidity3pm
	High: Class value: 0	Counts: [15092, 15715]
	Low: Class value: 0	Counts: [91789, 110321]
	Mid: Class value: 1	Counts: [12722, 19424]


REGLAS
IF Humidity3pm[High] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF Humidity3pm[Low] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF Humidity3pm[Mid] THEN RainTomorrow[Yes]
	AND aggregation function : fmin
	OR aggregation function  : fmax

CPU times: total: 2.06 s
Wall time: 2.06 s


### Árbol de Profundad 2

In [20]:
%%time 
t = C45(fuzzy_X.columns)
t.max_depth = 2
t.fit(fuzzy_X, y)
print("ARBOL")
print(t.tree)
print("")
print("")

print("REGLAS")
arbol = TreeRuleBasedSystem()
arbol.fit(t, fuzzy_vars, discrete_variables, target_variable)

ARBOL
Feature Humidity3pm
	High: Feature WindDir3pm
		E: Class value: 0	Counts: [3509, 3655]
		N: Class value: 0	Counts: [2671, 2719]
		S: Class value: 0	Counts: [4630, 5003]
		W: Class value: 0	Counts: [4282, 4338]
	Low: Feature Cloud3pm
		High: Class value: 0	Counts: [58336, 69580]
		Low: Class value: 0	Counts: [19492, 20837]
		Mid: Class value: 0	Counts: [13961, 19904]
	Mid: Feature RainToday
		No: Class value: 1	Counts: [7585, 9965]
		Yes: Class value: 1	Counts: [5137, 9459]


REGLAS
IF WindDir3pm[E] AND Humidity3pm[High] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF WindDir3pm[N] AND Humidity3pm[High] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF WindDir3pm[S] AND Humidity3pm[High] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : fmax

IF WindDir3pm[W] AND Humidity3pm[High] THEN RainTomorrow[No]
	AND aggregation function : fmin
	OR aggregation function  : f

### Árbol de Profundidad 3

In [21]:
%%time 
t = C45(fuzzy_X.columns)
t.max_depth = 3
t.fit(fuzzy_X, y)
print("ARBOL")
print(t.tree)
print("")
print("")

print("REGLAS")
arbol = TreeRuleBasedSystem()
arbol.fit(t, fuzzy_vars, discrete_variables, target_variable)

ARBOL
Feature Humidity3pm
	High: Feature WindDir3pm
		E: Feature WindGustSpeed
			High: Class value: 0	Counts: [2352, 2485]
			Low: Class value: 0	Counts: [1153, 1166]
			Mid: Class value: 0	Counts: [4, 4]
		N: Feature WindGustDir
			E: Class value: 0	Counts: [341, 359]
			N: Class value: 0	Counts: [566, 575]
			S: Class value: 0	Counts: [1673, 1690]
			W: Class value: 0	Counts: [91, 95]
		S: Feature WindGustSpeed
			High: Class value: 0	Counts: [3074, 3418]
			Low: Class value: 0	Counts: [1555, 1582]
			Mid: Class value: 1	Counts: [2, 3]
		W: Feature MinTemp
			High: Class value: 0	Counts: [3969, 4004]
			Low: Class value: 0	Counts: [200, 219]
			Mid: Class value: 0	Counts: [113, 115]
	Low: Feature Cloud3pm
		High: Feature RainToday
			No: Class value: 0	Counts: [48886, 55747]
			Yes: Class value: 0	Counts: [9450, 13833]
		Low: Feature Cloud9am
			High: Class value: 0	Counts: [5276, 5783]
			Low: Class value: 0	Counts: [2508, 2855]
			Mid: Class value: 0	Counts: [11708, 12199]
		Mid: 

### Árbol de Profunidad 4

In [22]:
%%time 
t = C45(fuzzy_X.columns)
t.max_depth = 4
t.fit(fuzzy_X, y)
print("ARBOL")
print(t.tree)
print("")
print("")

print("REGLAS")
arbol = TreeRuleBasedSystem()
arbol.fit(t, fuzzy_vars, discrete_variables, target_variable)

ARBOL
Feature Humidity3pm
	High: Feature WindDir3pm
		E: Feature WindGustSpeed
			High: Feature Sunshine
				High: Class value: 0	Counts: [1836, 1911]
				Mid: Class value: 0	Counts: [516, 574]
			Low: Feature RainToday
				No: Class value: 0	Counts: [1139, 1149]
				Yes: Class value: 0	Counts: [14, 17]
			Mid: Feature MinTemp
				High: Class value: 0	Counts: [3, 3]
				Mid: Class value: 0	Counts: [1, 1]
		N: Feature WindGustDir
			E: Feature WindGustSpeed
				Low: Class value: 0	Counts: [180, 184]
				Mid: Class value: 0	Counts: [161, 175]
			N: Feature WindSpeed9am
				Low: Class value: 0	Counts: [562, 569]
				Mid: Class value: 0	Counts: [4, 6]
			S: Feature Cloud3pm
				High: Class value: 0	Counts: [794, 806]
				Low: Class value: 0	Counts: [805, 807]
				Mid: Class value: 0	Counts: [74, 77]
			W: Feature Temp3pm
				High: Class value: 0	Counts: [64, 64]
				Mid: Class value: 0	Counts: [27, 31]
		S: Feature WindGustSpeed
			High: Feature Cloud3pm
				High: Class value: 0	Counts: [1

### Árbol de Profunidad 5

In [23]:
%%time 
t = C45(fuzzy_X.columns)
t.max_depth = 5
t.fit(fuzzy_X, y)
print("ARBOL")
print(t.tree)
print("")
print("")

print("REGLAS")
arbol = TreeRuleBasedSystem()
arbol.fit(t, fuzzy_vars, discrete_variables, target_variable)

ARBOL
Feature Humidity3pm
	High: Feature WindDir3pm
		E: Feature WindGustSpeed
			High: Feature Sunshine
				High: Feature Cloud3pm
					High: Class value: 0	Counts: [1175, 1229]
					Low: Class value: 0	Counts: [76, 84]
					Mid: Class value: 0	Counts: [585, 598]
				Mid: Feature WindDir9am
					E: Class value: 0	Counts: [291, 333]
					N: Class value: 0	Counts: [115, 121]
					S: Class value: 0	Counts: [61, 70]
					W: Class value: 0	Counts: [49, 50]
			Low: Feature RainToday
				No: Feature Cloud3pm
					High: Class value: 0	Counts: [669, 677]
					Low: Class value: 0	Counts: [414, 415]
					Mid: Class value: 0	Counts: [56, 57]
				Yes: Feature Temp9am
					High: Class value: 0	Counts: [14, 16]
					Mid: Class value: 1	Counts: [0, 1]
			Mid: Feature MinTemp
				High: Feature MaxTemp
					Mid: Class value: 0	Counts: [3, 3]
				Mid: Feature MaxTemp
					High: Class value: 0	Counts: [1, 1]
		N: Feature WindGustDir
			E: Feature WindGustSpeed
				Low: Feature Cloud3pm
					High: Class v

<h1><font color="#004D7F" size=5>CONCLUSION FINAL</font></h1>

Este nuevo enfoque de los arboles, a nuestro parecer, es mas interesante, permite ver de una maners mas natural y menos numerica la potencia de los sistemas difusos, son menos precisos a la hora de dar una respuesta exacta ya que juegan con funciones de pertenecia, pero el resultado final es un sistema mucho mas natural a la hora de interpretarlo para las personas, ya que es mas sencillo entender 'Bajo' 'Medio' o 'Alto' que números como 12, 7 y 19.4

Dicho esto y en base a los resultados obtenidos, en intervalos de tiempo bastante aceptables podemos obtener sistemas con un alto grado de informacion sobre el terreno que planteamos, se pueden deducir razonamientos interesantes de bases de datos de mas de miles de valores, en apenas pocos segundos.